In [1]:
import matplotlib.pyplot as plt
from simtk.openmm import app
from simtk import openmm, unit
from openeye import oechem
from sys import stdout
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import sys
import string

import lxml.etree as etree

from simtk.openmm.app import element as elem
from simtk.openmm.app import Topology

import os
import math
import copy
import re
import numpy
import random

from simtk import openmm, unit

import time

import itertools

from rdkit import Chem


import sys
import io
from IPython.nbformat import current
import openeye.oechem
import openeye.oeomega
import openeye.oequacpac

from openeye import oechem, oequacpac
from openeye.oechem import *
from openeye.oedepict import *

from openforcefield.typing.engines.smirnoff import forcefield_rdk
from openforcefield.typing.engines import smirnoff
#from openforcefield import *
from openforcefield.utils import *
# sys.path.insert(0, os.path.dirname(os.path.abspath(os.path.join('..'))) + "/openforcefield/utils/")
# #from utils import *
# from utils import get_data_filename, extractPositionsFromOEMol, generateTopologyFromOEMol, extractPositionsFromRDKMol

# sys.path.insert(0, os.path.dirname(os.path.abspath(os.path.join('..'))) + "/openforcefield/typing/engines/smirnoff/")
# from forcefield import *
# from forcefield_utils import *

/home/shuzhe/anaconda2/envs/oepython2/lib/python2.7/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
/home/shuzhe/anaconda2/envs/oepython2/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)


In [19]:
mol_filename = '/home/shuzhe/Documents/openforcefield/openforcefield/data/molecules/AlkEthOH_test_filt1_tripos.mol2'
mol = Chem.MolFromMol2File(mol_filename, sanitize = True, removeHs = False)

print Chem.MolToMolBlock(mol)
positions_rdk = extractPositionsFromRDKMol(mol)
#print positions

atoms = mol.GetAtoms()
print atoms[0].GetPropsAsDict()
#print list(atoms[0].GetPropNames())
atoms[0].SetDoubleProp("PartialCharge", 0.3)
print atoms[0].GetPropsAsDict()["PartialCharge"]
[float(a.GetPropsAsDict()["_TriposPartialCharge"]) if "_TriposPartialCharge" in a.GetPropsAsDict() else 0.0 for a in mol.GetAtoms()]

MiniDrugBank_141
     RDKit          3D

 12 12  0  0  0  0  0  0  0  0999 V2000
    0.7145    2.0625    0.0000 S   0  0  0  0  0  0  0  0  0  0  0  0
    0.7145    1.2375    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.4289    0.8250    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    1.4289    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.7145   -0.4125    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 C   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.8250    0.0000 N   0  0  0  0  0  0  0  0  0  0  0  0
    0.7145    2.0625    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.4289    0.8250    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    1.4289    0.0000    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    0.7145   -0.4125    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
    0.0000    0.0000    0.0000 H   0  0  0  0  0  0  0  0  0  0  0  0
  1  2  1  0
  2  3  2  0
  3  4  1  0
  4  5  2  0
  5  6  1  0
  6  7  2  0
 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [20]:

Chem.Draw.MolsToGridImage([mol],molsPerRow=1)


#print positions

# Load a SMIRNOFF small molecule forcefield for alkanes, ethers, and alcohols
forcefield = smirnoff.forcefield_rdk.ForceField("/home/shuzhe/Documents/openforcefield/openforcefield/data/forcefield/smirnoff99Frosst.ffxml")
#forcefield = ForceField('/home/shuzhe/Documents/openforcefield/openforcefield/data/forcefield/Frosst_AlkEtOH_parmAtFrosst.ffxml')

# Create the OpenMM system
topology_rdk = smirnoff.forcefield_rdk.generateTopologyFromRDKMol(mol)
system_rdk = forcefield.createSystem(topology_rdk, [mol])

#g1,g2,e1,e2 = smirnoff.forcefield_utils.compare_system_energies(topology, topology, system, system, positions)

In [24]:
mol_filename = '/home/shuzhe/Documents/openforcefield/examples/SMIRNOFF_simulation/AlkEthOH_r51.mol2'
mol_filename = '/home/shuzhe/Documents/openforcefield/openforcefield/data/molecules/AlkEthOH_test_filt1_tripos.mol2'

# Load molecule using OpenEye tools
mol = oechem.OEGraphMol()
ifs = oechem.oemolistream(mol_filename)
flavor = oechem.OEIFlavor_Generic_Default | oechem.OEIFlavor_MOL2_Default | oechem.OEIFlavor_MOL2_Forcefield
ifs.SetFlavor( oechem.OEFormat_MOL2, flavor)
oechem.OEReadMolecule(ifs, mol )
oechem.OETriposAtomNames(mol)

# Get positions in OpenMM-compatible format
positions_oe = extractPositionsFromOEMol(mol)
#print positions

# Load a SMIRNOFF small molecule forcefield for alkanes, ethers, and alcohols
forcefield = smirnoff.forcefield.ForceField("/home/shuzhe/Documents/openforcefield/openforcefield/data/forcefield/smirnoff99Frosst.ffxml")
#forcefield = ForceField('/home/shuzhe/Documents/openforcefield/openforcefield/data/forcefield/Frosst_AlkEtOH_parmAtFrosst.ffxml')

# Create the OpenMM system
topology_oe = smirnoff.forcefield.generateTopologyFromOEMol(mol)
system_oe = forcefield.createSystem(topology_oe, [mol])

g1,g2,e1,e2 = smirnoff.forcefield_utils.compare_system_energies(topology_oe, topology_rdk, system_oe, system_rdk, positions_oe, positions_rdk, skip_assert = True)


Num (type) 	 Num (type) 	 Num (type) 	 Num (type) 	 per 	 phase 	 k0
  4 ( C4)-   3 ( C3)- 	 2 ( C2)- 	   8 ( H2)- 	 2.000000 	 180.000000 	 15.167000 
  0 ( S1)-   1 ( C1)- 	 2 ( C2)- 	   8 ( H2)- 	 2.000000 	 180.000000 	 15.167000 
  8 ( H2)-   2 ( C2)- 	 3 ( C3)- 	   9 ( H3)- 	 2.000000 	 180.000000 	 15.167000 
  1 ( C1)-   2 ( C2)- 	 3 ( C3)- 	   9 ( H3)- 	 2.000000 	 180.000000 	 15.167000 
  2 ( C2)-   3 ( C3)- 	 4 ( C4)- 	   5 ( C5)- 	 2.000000 	 180.000000 	 15.167000 
  3 ( C3)-   4 ( C4)- 	 5 ( C5)- 	  11 ( H5)- 	 2.000000 	 180.000000 	 15.167000 
  2 ( C2)-   3 ( C3)- 	 4 ( C4)- 	  10 ( H4)- 	 2.000000 	 180.000000 	 15.167000 
  0 ( S1)-   1 ( C1)- 	 6 ( N1)- 	   5 ( C5)- 	 2.000000 	 180.000000 	 12.552000 
  5 ( C5)-   4 ( C4)- 	 3 ( C3)- 	   9 ( H3)- 	 2.000000 	 180.000000 	 15.167000 
  0 ( S1)-   1 ( C1)- 	 2 ( C2)- 	   3 ( C3)- 	 2.000000 	 180.000000 	 15.167000 
  2 ( C2)-   1 ( C1)- 	 0 ( S1)- 	   7 ( H1)- 	 2.000000 	 180.000000 	 1.393272 
 10 ( H4)-   4 ( C4

In [28]:
print topology_oe._reference_molecules

AttributeError: 'Topology' object has no attribute '_reference_molecules'

In [22]:
rdk_mol = Chem.MolFromSmiles('c1ccnc(c1)S')
rdk_mol.GetAtoms()[0].GetPropsAsDict()


def compare_system_energies( topology0, topology1, system0, system1, positions0, positions1=None, label0="AMBER system", label1 = "SMIRNOFF system", verbose = True, skip_assert = False, skip_improper = False ):
    """
    Given two OpenMM systems, check that their energies and component-wise
    energies are consistent, and return these. The same positions will be used
    for both systems unless a second set of positions is provided.

    Parameters
    ----------
    topology0 : OpenMM Topology
        Topology of first system
    topology1 : OpenMM Topology
        Topology of second system
    system0 : OpenMM System
        First system for comparison (usually from AMBER)
    system1 : OpenMM System
        Second system for comparison (usually from SMIRNOFF)
    positions0 : simtk.unit.Quantity wrapped
        Positions to use for energy evaluation comparison
    positions1 (optional) : simtk.unit.Quantity wrapped (optional)
        Positions to use for second OpenMM system; original positions are used
        if this is not provided
    label0 (optional) : str
        String labeling system0 for output. Default, "AMBER system"
    label1 (optional) : str
        String labeling system1 for output. Default, "SMIRNOFF system"
    verbose (optional) : bool
        Print out info on energies, True/False (default True)
    skip_assert (optional) : bool
        Skip assertion that energies must be equal within specified tolerance. Default False.
    skip_improper (optional) : bool
        Skip detailed checking of force terms on impropers (helpful here if comparing with AMBER force fields using different definitions of impropers.) Default False.

    Returns
    ----------
        groups0 : dict
            As returned by openmoltools.system_checker.check_energy_groups,
            a dictionary with keys "bond", "angle", "nb", "torsion" and values
            corresponding to the energies of these components for the first simulation object
        groups1 : dict
            As returned by openmoltools.system_checker.check_energy_groups,
            a dictionary with keys "bond", "angle", "nb", "torsion" and values
            corresponding to the energies of these components for the second simulation object
        energy0 : simtk.unit.Quantity
            Energy of first system
        energy1 : simtk.unit.Quantity
            Energy of second system

    TO DO:
        Allow energy extraction/comparison of terms specified by particular
        SMARTS queries i.e. for specific bond, angle, or torsional terms.
    """

    # Create integrator
    timestep = 1.0 * unit.femtoseconds
    integrator0 = simtk.openmm.VerletIntegrator( timestep )
    integrator1 = simtk.openmm.VerletIntegrator( timestep )

    # Grab second positions
    if positions1 == None:
        positions1 = copy.deepcopy( positions0 )

    # Create simulations
    platform = simtk.openmm.Platform.getPlatformByName("Reference")
    simulation0 = app.Simulation( topology0, system0, integrator0, platform = platform )
    simulation0.context.setPositions(positions0)
    simulation1 = app.Simulation( topology1, system1, integrator1, platform = platform )
    simulation1.context.setPositions(positions1)

    # Print what torsions were found if verbose
    if verbose:
        # Build list of atoms for debugging info
        atoms0 = [ atom for atom in simulation0.topology.atoms() ]
        atoms1 = [ atom for atom in simulation1.topology.atoms() ]
        # Loop over first system and print torsion info
        for force in simulation0.system.getForces():
            if type(force) == mm.PeriodicTorsionForce:
                print("Num (type) \t Num (type) \t Num (type) \t Num (type) \t per \t phase \t k0")
                for k in range(force.getNumTorsions()):
                    i0, i1, i2, i3, per, phase, k0 = force.getTorsionParameters(k)
                    print("%3s (%3s)- %3s (%3s)- \t %s (%3s)- \t %3s (%3s)- \t %f \t %f \t %f " % (i0, atoms0[i0].name, i1, atoms0[i1].name, i2, atoms0[i2].name, i3, atoms0[i3].name, per, phase/unit.degree, k0/unit.kilojoule_per_mole) )
        for force in simulation1.system.getForces():
            if type(force) == mm.PeriodicTorsionForce:
                print("Num (type) \t Num (type) \t Num (type) \t Num (type) \t per \t phase \t k0")
                for k in range(force.getNumTorsions()):
                    i0, i1, i2, i3, per, phase, k0 = force.getTorsionParameters(k)
                    print("%3s (%3s)- %3s (%3s)- %3s (%3s)- %3s (%3s) - %f \t %f \t %f " % (i0, atoms1[i0].name, i1, atoms1[i1].name, i2, atoms1[i2].name, i3, atoms1[i3].name, per, phase/unit.degree, k0/unit.kilojoule_per_mole) )